<a href="https://colab.research.google.com/github/alexshultz/comfyui-colab/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
# Install xFormers for CUDA 12.4 first
!pip install xformers!=0.0.18 --index-url https://download.pytorch.org/whl/cu124
# Then install remaining requirements
!pip install -r requirements.txt

# InsightFace and onnxruntime
!echo -= Installing InsightFace and onnxruntime for IP-Adapter FaceID =-
!pip install insightface onnxruntime-gpu

# Update or install IP-Adapter custom nodes
!echo -= Updating ComfyUI_IPAdapter_plus =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_IPAdapter_plus && git pull || git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ComfyUI_IPAdapter_plus)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt; else echo "requirements.txt missing, installing manually" && pip install insightface opencv-python-headless numpy torch torchvision; fi

# Update or install ComfyUI_essentials custom nodes
!echo -= Updating ComfyUI_essentials =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_essentials && git pull || git clone https://github.com/cubiq/ComfyUI_essentials.git ComfyUI_essentials)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt; else echo "No additional requirements for ComfyUI_essentials"; fi


# Update or install Comfy_Dungeon custom nodes
# https://github.com/cubiq/Comfy_Dungeon
!echo -= Updating Comfy_Dungeon =-
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd Comfy_Dungeon && git pull || git clone https://github.com/cubiq/Comfy_Dungeon.git Comfy_Dungeon)
# No requirements.txt specified in README, assuming base ComfyUI deps suffice

# Install huggingface model dowloader
!echo -= huggingface_hub =-
!pip install huggingface_hub


!echo -= Done =-


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

Great tools in the AP-Adapter package
`https://github.com/cubiq/ComfyUI_IPAdapter_plus?tab=readme-ov-file`

In [ ]:
# Checkpoints

from huggingface_hub import snapshot_download

# Download ProteusV0.3
print("Downloading ProteusV0.3...")
snapshot_download(repo_id="dataautogpt3/ProteusV0.3", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/ProteusV0.3", ignore_patterns=["*.git*"])

# Download dreamshaper-xl-v2-turbo
print("Downloading dreamshaper-xl-v2-turbo...")
snapshot_download(repo_id="Lykon/dreamshaper-xl-v2-turbo", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/dreamshaper-xl-v2-turbo", ignore_patterns=["*.git*"])

# Download LCM-LoRA for SDXL
print("Downloading lcm-lora-sdxl...")
!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors -O /content/drive/MyDrive/ComfyUI/models/loras/lcm-lora-sdxl.safetensors
print("All downloads completed.")

### SDXL
### Recommended workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/
# Download SD-XL 1.0-base
print("Downloading SD-XL 1.0-base...")
snapshot_download(repo_id="stabilityai/stable-diffusion-xl-base-1.0", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/SD-XL_1.0-base", ignore_patterns=["*.git*"])

## SDXL ReVision
# Download clip_vision_g
print("Downloading clip_vision_g...")
snapshot_download(repo_id="comfyanonymous/clip_vision_g", local_dir="/content/drive/MyDrive/ComfyUI/models/clip_vision/clip_vision_g", ignore_patterns=["*.git*"])


### SD1.5
# Download Stable Diffusion v1.5
print("Downloading Stable Diffusion v1.5...")
snapshot_download(repo_id="Comfy-Org/stable-diffusion-v1-5-archive", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/Stable_Diffusion_v1.5", ignore_patterns=["*.git*"])

# some SD1.5 anime style
# Download OrangeMixs
print("Downloading OrangeMixs...")
snapshot_download(repo_id="WarriorMama777/OrangeMixs", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/OrangeMixs", ignore_patterns=["*.git*"])

# Download Anything V3
# gated ... need to figure out how to show i have permission
#print("Downloading Anything V3...")
#snapshot_download(repo_id="Linaqruf/anything-v3.0", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/Anything_V3", ignore_patterns=["*.git*"])

## IP Adapter
# Download IP-Adapter
print("Downloading IP-Adapter...")
snapshot_download(repo_id="h94/IP-Adapter", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/IP-Adapter", ignore_patterns=["*.git*"])

# Download IP-Adapter-FaceID
# FaceID models require insightface, you need to install it in your ComfyUI environment. Check this issue for help. Remember that most FaceID models also need a LoRA.
print("Downloading IP-Adapter-FaceID...")
snapshot_download(repo_id="h94/IP-Adapter-FaceID", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/IP-Adapter-FaceID", ignore_patterns=["*.git*"])

## NSFW
# Download NSFWmodel
print("Downloading NSFWmodel...")
snapshot_download(repo_id="lexa862/NSFWmodel", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/NSFWmodel", ignore_patterns=["*.git*"])

##UnCLIP
# Download illuminatiDiffusionV1_v11_unCLIP
print("Downloading illuminatiDiffusionV1_v11_unCLIP...")
snapshot_download(repo_id="comfyanonymous/illuminatiDiffusionV1_v11_unCLIP", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/illuminatiDiffusionV1_v11_unCLIP", ignore_patterns=["*.git*"])
# Download wd-1.5-beta2_unCLIP
print("Downloading wd-1.5-beta2_unCLIP...")
snapshot_download(repo_id="comfyanonymous/wd-1.5-beta2_unCLIP", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/wd-1.5-beta2_unCLIP", ignore_patterns=["*.git*"])

##VAE
# Download sd-vae-ft-mse-original
print("Downloading sd-vae-ft-mse-original...")
snapshot_download(repo_id="stabilityai/sd-vae-ft-mse-original", local_dir="/content/drive/MyDrive/ComfyUI/models/vae/sd-vae-ft-mse-original", ignore_patterns=["*.git*"])
# Download Waifu Diffusion v1.4 (anime)
print("Downloading Waifu Diffusion v1.4...")
snapshot_download(repo_id="hakurei/waifu-diffusion-v1-4", local_dir="/content/drive/MyDrive/ComfyUI/models/checkpoints/Waifu_Diffusion_v1.4", ignore_patterns=["*.git*"])


### CLIPVision model (needed for styles model)
# Download clip-vit-large-patch14
print("Downloading clip-vit-large-patch14...")
snapshot_download(repo_id="openai/clip-vit-large-patch14", local_dir="/content/drive/MyDrive/ComfyUI/models/clip_vision/clip-vit-large-patch14", ignore_patterns=["*.git*"])


### ControlNet
# Download ControlNet-v1-1_fp16_safetensors
print("Downloading ControlNet-v1-1_fp16_safetensors...")
snapshot_download(repo_id="openai/clip-vit-large-patch14", local_dir="/content/drive/MyDrive/ComfyUI/models/controlnet/clip-vit-large-patch14", ignore_patterns=["*.git*"])
# Download ontrol-LoRA Model
print("Downloading Control-LoRA Model...")
snapshot_download(repo_id="stabilityai/control-lora", local_dir="/content/drive/MyDrive/ComfyUI/models/controlnet/Control-LoRA_Model", ignore_patterns=["*.git*"])

### Loras
# Download required SDXL LCM LoRA model (rename as required by Comfy_Dungeon)
print("Downloading Latent Consistency Model (LCM) LoRA: SDXL...")
snapshot_download(repo_id="latent-consistency/lcm-lora-sdxl", local_dir="/content/drive/MyDrive/ComfyUI/models/loras/Latent_Consistency_Model_(LCM)_LoRA_SDXL", ignore_patterns=["*.git*"])


### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

# SDXL ReVision
#!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P ./models/clip_vision/

# SD1.5
#!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -P ./models/checkpoints/

# Dreamshaper 8 (for SD1.5)
!wget -c https://civitai.com/api/download/models/128713 -O ./models/checkpoints/dreamshaper_8.safetensors

## IP-Adapter
# IP-Adapter Models
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -P ./models/ipadapter/
# Basic model, average strength
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter/
# Light impact model
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -P ./models/ipadapter/
# Plus model, very strong
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -P ./models/ipadapter/
# Face model, portraits
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P ./models/ipadapter/
# Stronger face model, not necessarily better
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -P ./models/ipadapter/
# Base model, requires bigG clip vision encoder
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -P ./models/ipadapter/
# SDXL model
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL plus model
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL face model
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -P ./models/ipadapter/
# vit-G SDXL model, requires bigG clip vision encoder
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P ./models/ipadapter/

# IP-Adapter - FaceID models require insightface, you need to install it in your ComfyUI environment. Check this issue for help. Remember that most FaceID models also need a LoRA.
# base FaceID model
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -P ./models/ipadapter/
# FaceID plus v2
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -P ./models/ipadapter/
# text prompt style transfer for portraits
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait-v11_sd15.bin -P ./models/ipadapter/
# SDXL base FaceID
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin -P ./models/ipadapter/
# SDXL plus v2
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin -P ./models/ipadapter/
# SDXL text prompt style transfer
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl.bin -P ./models/ipadapter/
# very strong style transfer SDXL only
#!wget -c ip-adapter-faceid-portrait_sdxl_unnorm.bin -P ./models/ipadapter/
# IP-Adapter - community - general composition ignoring style and content, more about it at https://huggingface.co/ostris/ip-composition-adapter
#!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sd15.safetensors -P ./models/ipadapter/
# SDXL version
#!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sdxl.safetensors -P ./models/ipadapter/

# IP-Adapter Loras
# IP-Adapter - FaceID - Most FaceID models require a LoRA. If you use the IPAdapter Unified Loader FaceID it will be loaded automatically if you follow the naming convention. Otherwise you have to load them manually, be careful each FaceID model has to be paired with its own specific LoRA.
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors -P ./models/loras/
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -P ./models/loras/
# SDXL FaceID LoRA
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors -P ./models/loras/
# SDXL plus v2 LoRA
#!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors -P ./models/loras/

# IP-Adapter CLIPVision models
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
#!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors


# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
#!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
#!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora

# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
#!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin

# ControlNet
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

# ControlNet SDXL
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/


# Download and extract InsightFace buffalo_l models
!echo -= Downloading InsightFace buffalo_l models =-
!wget -c https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip
!unzip -o /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip -d /content/drive/MyDrive/ComfyUI/models/insightface/
!rm /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip


# Download required SDXL LCM LoRA model (rename as required by Comfy_Dungeon)
#!echo -= Downloading SDXL LCM LoRA for Comfy_Dungeon =-
#!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors -O /content/drive/MyDrive/ComfyUI/models/loras/lcm-lora-sdxl.safetensors

!echo -= Done =-


### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server